STATISTIQUE
===
---
Librairie
---



Connaitre le nombre d'image disponible sur le site d'étude.

**Importation des librairies necessaire au traitement.**
- json 
- os
- requests
- time
- geojsonsio

après l'importation de librairie entré votre compte API de Planetlabs ([trouvé son API key][API])

l'ensemble de l'information provient de ce site.

https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/planet_data_api_introduction.ipynb

[API]:https://developers.planet.com/planetschool/getting-started/

In [ ]:
# import
import time
import os
import geojsonio
import json
import requests

os.environ['PL_API_KEY']='' # Entré votre API_KEY personnel.
PLANET_API_KEY= os.getenv ('PL_API_KEY')

Les prochaines étapes permet à la vérification de la connection entre l'utilisateur et le serveur de Planetlabs.

In [ ]:
def p(data):
    print(json.dumps(data, indent=2))

In [ ]:
# définir l'URL d'accès au donnée de planetlab
URL = "https://api.planet.com/data/v1"

session = requests.Session()

session.auth = (PLANET_API_KEY, "")

In [ ]:
res = session.get(URL)

In [ ]:
res.status_code # 200 veut dire que la requête est correct alor que 402 non

In [ ]:
res.text

In [ ]:
p(res.json())

In [ ]:
#validation des requêtes et sélection du links
print(res.json()["_links"]["item-types"])


Félicitation! vous venez de faire votre première requête à Planet Data API en utilisant Python

Maintenant, on va prendre le temps de regarder comment avoir accès à la panoplie d'image de planet.

In [ ]:
# La fonction stats permet de voir le nombre d'image de disponible selon nos filtres.

stats_url = "{}/stats".format(URL)

print(stats_url)

In [ ]:
# Sélection du capteur

item_types = ["REOrthoTile", "PSScene4Band"]

# exemple (item_types = ["PSScene4Band"])


Voici les noms des différent capteur et de leur base de donné

Il est possible de trouvé les différents critère et écriture sur [les item-assets][item].

[item]:https://developers.planet.com/docs/api/items-assets/

Les filtres 
---

La prochaine étaple est d'attribuer des filtres à notre recherche

Elle peut être sur le temps, la position géographique, la couverture nuageuse, etc... On peut même les combinées

Vous pouvez d'autre exemple [ici][Filter]

[Filter]:https://developers.planet.com/docs/api/searches-filtering/

In [ ]:
date_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config":{
        "gte":"2016-01-01T00:00:00.000Z",
    }
}
# isle verte
geom_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config":{
        "type": "Polygon",
        "coordinates":[
          [
            [
              -69.51187133789062,
              47.958663127446556
            ],
            [
              -69.44320678710938,
              47.958663127446556
            ],
            [
              -69.44320678710938,
              48.004625021133904
            ],
            [
              -69.51187133789062,
              48.004625021133904
            ],
            [
              -69.51187133789062,
              47.958663127446556
            ]
          ]
        ]
    }
}

In [ ]:
# le and_filter permet de jumelée des filtres entre eux afin d'y faire la request complete.
and_filter ={
    "type" : "AndFilter",
    "config": [date_filter, geom_filter,]
}
#p(and_filter)

In [ ]:
# faire une request et imprimé le nombre de features de disponible.
request = {
    "item_types": item_types,
    "interval": "year",
    "filter": and_filter
}

res = session.post(stats_url, json=request)

p(res.json())

voila ce qui complet un demande quantitative du nombre d'image disponible sur notre secteur!!! 

Acquisition des imageries
---



Quick-search
---
---


L'execuotion des recherche se fait à partir du **quick-search**

In [ ]:
quick_url = "{}/quick-search".format(URL)

p(quick_url)

In [ ]:
# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)
# Assign the response to a variable
geojson = res.json()
# Print the response
p(geojson)

**Le nombre d'images de disponible**

Il est possible de connaitre le nombre images trouvé avec le code suivant.

In [ ]:
# Assign a features variable 
features = geojson["features"]
# Get the number of features present in the response
len(features)# 250 est le nombre de features que le notebook peut affiché 

il est important de crée une loop pour afficher l'ensemble des images. 

In [ ]:
# Loop over all the features from the response
for f in features:
    # Print the ID for each feature
    p(f["id"])

In [ ]:
# afficher la première image seulement.
p(features[0])

In [ ]:
# Assign the "_links" -> "_next" property (link to next page of results) to a variable 
next_url = geojson["_links"]["_next"]

# Print the link to the next page of results
print(next_url)

The page size can be set with a _page_size parameter in the request:

Le nombre d'image par page peut être choisir avec le paramettre **_page _size** dans la requête.



In [ ]:
# Send the POST request to the API quick search endpoint with a page size of 9
res = session.post(quick_url, json=request, params={"_page_size" : 10})

# Assign the response to a variable
geojson = res.json()

# Get the number of features present in the response
len(geojson["features"])

### Mapping Footprints Example

 ce petit bout de code permet de passée de page en page que nous avons crée précédement passée à la page suivente 

In [ ]:
# Assign the url variable to display the geojsonio map
url = geojsonio.display(res.text)

In [ ]:
# Assign the next_url variable to the next page of results from the response (Setup the next page of results)
next_url = geojson["_links"]["_next"]

# Get the next page of results
res = session.get(next_url)

# Assign the response to a variable
geojson = res.json()

# Get the url see results on geojson.io
url = geojsonio.to_geojsonio(res.text)

### Loop pour le traitement de plusieurs images 

Les étapes individuelle suivront après la loop.

---


In [ ]:
# Assign a variable to the search result features (items)
features = geojson["features"]
#p(feature)
for i in range(len(features)):
    # Get the first result's feature
    feature = features[i] 
    # Print the ID
    p(feature["id"])
    # Get the assets link for the item
    assets_url = feature["_links"]["assets"]
    # Print the assets link
    print(assets_url)
    # Send a GET request to the assets url for the item (Get the list of available assets for the item)
    res = session.get(assets_url)
    # Assign a variable to the response
    assets = res.json()
    # Print the asset types that are available
    print(assets.keys())
    # Assign a variable to the visual asset from the item's assets
    visual = assets["analytic"]
    # Print the visual asset data    
    p(visual)
    # Setup the activation url for a particular asset (in this case the visual asset)
    activation_url = visual["_links"]["activate"]

    # Send a request to the activation url to activate the item
    res = session.get(activation_url)

    # Print the response from the activation request
    p(res.status_code)
    # Print the visual asset data
    p(visual)
    import time
    
    asset_activated = False
    
    while asset_activated == False:
        # Send a request to the item's assets url
        res = session.get(assets_url)

        # Assign a variable to the item's assets url response
        assets = res.json()

        # Assign a variable to the visual asset from the response
        visual = assets["analytic"]

        asset_status = visual["status"]
    
        # If asset is already active, we are done
        if asset_status == 'active':
            asset_activated = True
            print("Asset is active and ready to download")
            
    # Assign a variable to the visual asset's location endpoint
    location_url = visual["location"]

    # Print the location endpoint
    #print(location_url)
    
    # Create a function to download asset files
    # Parameters: 
    # - url = (the location url)
    # - filename (the filename to save it as. defaults to whatever the file is called originally)

    def pl_download(url, filename=None):
    
        # Send a GET request to the provided location url, using your API Key for authentication
        res = requests.get(url, stream=True, auth=(PLANET_API_KEY, ""))
        # If no filename argument is given
        if not filename:
            # Construct a filename from the API response
            if "content-disposition" in res.headers:
                filename = res.headers["content-disposition"].split("filename=")[-1].strip("'\"")
            # Construct a filename from the location url
            else:
                filename = url.split("=")[1][:10]
        # Save the file
        with open('D:/Planet/' + filename, "wb") as f:
            for chunk in res.iter_content(chunk_size=1024):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    f.flush()

        return filename
    
    # Download the file from an activated asset's location url
    pl_download(location_url)

print ("téléchargement complet")    
# Print the permissions
#p(features["_permissions"])

### Étape de l'activation au téléchargement

#### Activation asset

In [ ]:
# Assign a variable to the search result features (items)
features = geojson["features"]

# Get the first result's feature
feature = features[0]

# Print the ID
p(feature["id"])

# Print the permissions
p(feature["_permissions"])

In [ ]:
# Assign a variable to the search result features (items)
features = geojson["features"]

In [ ]:
# Get the assets link for the item
assets_url = feature["_links"]["assets"]

# Print the assets link
print(assets_url)

In [ ]:
# Send a GET request to the assets url for the item (Get the list of available assets for the item)
res = session.get(assets_url)

# Assign a variable to the response
assets = res.json()

In [ ]:
# Print the asset types that are available
print(assets.keys())

In [ ]:
# Assign a variable to the visual asset from the item's assets
visual = assets["analytic"]

# Print the visual asset data
p(visual)

activation


In [ ]:
# Setup the activation url for a particular asset (in this case the visual asset)
activation_url = visual["_links"]["activate"]

# Send a request to the activation url to activate the item
res = session.get(activation_url)

# Print the response from the activation request
p(res.status_code)

In [ ]:
import time

asset_activated = False

while asset_activated == False:
    # Send a request to the item's assets url
    res = session.get(assets_url)

    # Assign a variable to the item's assets url response
    assets = res.json()

    # Assign a variable to the visual asset from the response
    visual = assets["analytic"]

    asset_status = visual["status"]
    
    # If asset is already active, we are done
    if asset_status == 'active':
        asset_activated = True
        print("Asset is active and ready to download")

# Print the visual asset data    
p(visual)

## Téléchargement 

### avec lien URL
le code si dessous donne une url pour le téléchargement manuelle

In [ ]:
# Assign a variable to the visual asset's location endpoint
location_url = visual["location"]

# Print the location endpoint
print(location_url)

### Automatique

In [ ]:
# Create a function to download asset files
# Parameters: 
# - url (the location url)
# - filename (the filename to save it as. defaults to whatever the file is called originally)

def pl_download(url, filename=None):
    
    # Send a GET request to the provided location url, using your API Key for authentication
    res = requests.get(url, stream=True, auth=(PLANET_API_KEY, ""))
    # If no filename argument is given
    if not filename:
        # Construct a filename from the API response
        if "content-disposition" in res.headers:
            filename = res.headers["content-disposition"].split("filename=")[-1].strip("'\"")
        # Construct a filename from the location url
        else:
            filename = url.split("=")[1][:10]
    # Save the file
    with open('output/' + filename, "wb") as f:
        for chunk in res.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()

    return filename

In [ ]:
# Download the file from an activated asset's location url
pl_download(location_url)

### Rate Limits
Now that you know how to download assets, you should probably keep some of the API rate limits in mind:

---
- For most endpoints, the rate limit is 10 requests per second, per API key.
- For activation endpoints, the rate limit is 5 requests per second, per API key.
- For download endpoints, the rate limit is 15 requests per second, per API key.
- If you're writing to code to automate accessing the API, you should account for 429 responses and handle retries after a backoff period.

## Saved Searches
---


The /searches endpoint lets you created saved searches that can be reused.

To view your saved searches, visit the searches/?search_type=saved endpoint.

Finally, let's create a saved search:

**La saugarde à besoin des traitements de téléchargement s'il l'ont veux sauvegardé quelque chose**

In [ ]:
# Setup the saved searches endpoint url
searches_url = "{}/searches".format(URL)

In [ ]:
# Setup the request, specifying a name for the saved search, along with the usual search item_types and filter.
request = {
    "name" : "PManicouagan",
    "item_types" : item_types,
    "filter" : and_filter
}

# Send a POST request to the saved searches endpoint (Create the saved search)
res = session.post(searches_url, json=request)

# Print the response
p(res.json())

Now that we created a saved search, let's get a list of our saved searches:

In [ ]:
# Send a GET request to the saved searches endpoint with the saved search type parameter (Get saved searches)
res = session.get(searches_url, params={"search_type" : "saved"})

# Assign a variable to the searches property in the saved searches response
searches = res.json()["searches"]
print(searches[0])#['name'])
print(len(searches))

# Loop through the searches
for search in searches:
    # Print the ID, Created Time, and Name for each saved search
    print("id: {} created: {} name: {}".format(search["id"], search["created"], search['name']))


Ok, now let's check out the results from a particular saved search:

In [ ]:
# Setup the saved search url, using the first saved search in the list
saved_url = "{}/{}".format(searches_url, searches[0]["id"])

# Print the saved search url
p(saved_url)

# Setup the saved search's results url
results_url = "{}/results".format(saved_url)

# Print the saved search's results url
p(results_url)

In [ ]:
# Send a GET request to the saved search url (Get the saved search data)
res = session.get(saved_url)

# Print the response
p(res.json())

In [ ]:

# Send a GET request to the saved search results url (Get the saved search results)
results = session.get(results_url).json()

# Print the number of features in the saved search
print(len(results["features"]))

# Print the first feature in the saved search
print(results["features"][0])

A saved search can also be updated by senidng a PUT request with a changed search definition back to the API.

Did you know that saved searches can also send you a daily email to alert you to when new items are added to the search resutlts? Oh yeah!

In [ ]:
# Assign a variable to the saved search response
search = res.json()

# Print the saved search
p(search)

**Changement le nom de la sauvegarde et l'envoyé  par courriel**

In [ ]:
# Change the saved search name to "South Vancouver Island"
search["name"] = "PManicouagan"

# Set the daily email enabled to true (Get email alerts when new items show up in this search)
search["__daily_email_enabled"] = True

# Send a PUT request to the saved search endpoint (Update the saved search)
res = session.put(saved_url, json=search)

# The response status code
res.status_code

## Conclusion
You made it! You should now be able to use the Planet API to find items and assets by searching, get stats and save searches, and activate and download assets using Python code!

Don't forget to visit the Planet API Reference Docs.